In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import os
import cv2
import numpy as np
import pywt
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC

# Function to load and preprocess images from multiple folders
def load_and_preprocess_images(folder_paths, label):
    images = []
    labels = []
    for folder_path in folder_paths:
        for filename in os.listdir(folder_path):
            if filename.endswith(".jpg"):
                image_path = os.path.join(folder_path, filename)
                img = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
                img = cv2.resize(img, (64, 64))  # Resize to a common size
                images.append(img)
                labels.append(label)
    return images, labels

# Define folder paths for pedestrian and non-pedestrian images
pedestrian_folders = [
    "/content/drive/MyDrive/ML1/JPEGImagesTest",
    "/content/drive/MyDrive/ML1/JPEGImagesTrain",
    # Add more pedestrian folders as needed
]

non_pedestrian_folders = [
    "/content/drive/MyDrive/ML1/JPEGImagesTest",
    # Add more non-pedestrian folders as needed
]

# Load and preprocess images from all pedestrian and non-pedestrian folders
pedestrian_images, pedestrian_labels = load_and_preprocess_images(pedestrian_folders, label=1)
non_pedestrian_images, non_pedestrian_labels = load_and_preprocess_images(non_pedestrian_folders, label=0)

# Combine pedestrian and non-pedestrian images and labels
X = np.concatenate([pedestrian_images, non_pedestrian_images], axis=0)
y = np.concatenate([pedestrian_labels, non_pedestrian_labels], axis=0)

# Extract DWT features from the combined images
def extract_dwt_features(images):
    dwt_features = []
    for img in images:
        # Perform DWT on the image
        coeffs = pywt.dwt2(img, 'bior1.3')  # Use the Biorthogonal 1.3 wavelet
        LL, (LH, HL, HH) = coeffs  # Approximation and detail coefficients

        # Flatten and concatenate the coefficients
        features = np.concatenate((LL.ravel(), LH.ravel(), HL.ravel(), HH.ravel()))

        dwt_features.append(features)

    return np.array(dwt_features)

# Extract DWT features from the combined images
X_dwt = extract_dwt_features(X)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_dwt, y, test_size=0.2, random_state=42)

# Create an SVM classifier and train it
svm_model = SVC(kernel='linear')
svm_model.fit(X_train, y_train)

# Evaluate the model
accuracy = svm_model.score(X_test, y_test)
print(f"Accuracy: {accuracy * 100:.2f}%")


Accuracy: 71.38%
